In [ ]:
# pip install sentence_transformers

In [60]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json
import pickle

In [161]:
df = pd.read_csv('final_dataset_0.csv')

In [162]:
## load the embeddings pkl file
with open('final_dataset_embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

In [163]:
df['embeddings'] = embeddings

In [140]:
# Load the embeddings model
model = SentenceTransformer('all-MiniLM-L6-v2')

Recommendation using Sentence Transformers

In [165]:
def get_recommendations(query, data=df, top_n=10):
    # Compute the embedding for the query
    query_embedding = model.encode(query)

    # Calculate similarities and get top-N
    sim_scores = data['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x]).flatten()[0])
    top_indices = sim_scores.nlargest(top_n).index

    return data.iloc[top_indices]

In [166]:
def get_json_recommendations(query, df, top_n):
    res = get_recommendations(query, df, top_n=10)
    res = res.reset_index()
    res = res.loc[:, ['article_id', 'prod_name', 'price']]
    ## convert to json
    res = res.to_json(orient='records')
    return res

In [174]:
query = "baby dress"
## convert string to json
get = json.loads(get_json_recommendations(query, df, 10))

In [175]:
print(get)

[{'article_id': 814091002, 'prod_name': 'Nuria dress', 'price': 3999}, {'article_id': 809752002, 'prod_name': 'Doris dress', 'price': 3799}, {'article_id': 722911001, 'prod_name': 'Donna tassel dress', 'price': 3299}, {'article_id': 749513001, 'prod_name': 'HULDA dress - TVP', 'price': 5749}, {'article_id': 814091003, 'prod_name': 'Nuria dress', 'price': 9599}, {'article_id': 635040002, 'prod_name': 'GUGGENHEIM knitted dress', 'price': 9299}, {'article_id': 723652001, 'prod_name': 'BARBRO dress DENIM', 'price': 5349}, {'article_id': 695682001, 'prod_name': 'FLEUR dress', 'price': 6699}, {'article_id': 905853002, 'prod_name': 'Naomi fancy dress', 'price': 9149}, {'article_id': 629824002, 'prod_name': 'POLLY daycare dress TVP', 'price': 6949}]
